# Hough Transform Analysis - OOP Version

This notebook contains the complete Object-Oriented Programming refactored version of the Hough Transform Analysis.

## Architecture Overview

The code follows enterprise design patterns:
- **Domain Layer**: Core business entities (Particle, Peak, Track, HoughSquare)
- **Strategy Pattern**: Interchangeable easing algorithms
- **Repository Pattern**: Data access abstraction
- **Service Layer**: Business logic (PeakDetector, TrackSlicer, HoughMatcher, Visualizer)
- **Facade Pattern**: Simplified high-level API
- **Observer Pattern**: Progress tracking
- **Builder Pattern**: Flexible configuration

## How to Use

1. Run all cells sequentially (Cell → Run All)
2. Modify configuration in the last cell if needed
3. Execute the analysis

## 1. Imports and Setup

In [ ]:
# Standard library imports
import gc
import math
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Type, Iterator

# Scientific computing
import numpy as np
import pandas as pd
from numpy.lib.stride_tricks import sliding_window_view
from scipy.ndimage import gaussian_filter
from scipy.spatial.distance import cdist

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# ROOT file handling
import uproot

print("✓ All imports successful")

## 2. Domain Models - Particle

In [ ]:
@dataclass(frozen=True)
class Particle:
    """Immutable value object representing a particle."""
    pdg_id: int
    charge: float
    pt: float
    eta: float
    phi: float
    pz: float
    vz: float

    @property
    def pz_over_pt(self) -> float:
        return self.pz / self.pt if self.pt != 0 else 0.0

    @property
    def cotangent(self) -> float:
        return self.pz_over_pt

    def is_charged(self) -> bool:
        return self.charge != 0


class ParticleChargeRegistry:
    """Singleton registry for PDG particle ID to charge mappings."""
    
    _instance: Optional['ParticleChargeRegistry'] = None
    
    _CHARGE_MAP: Dict[int, float] = {
        # Gauge bosons
        22: 0, 23: 0, 24: 1, -24: -1, 21: 0,
        # Leptons
        11: -1, -11: 1, 12: 0, -12: 0,
        13: -1, -13: 1, 14: 0, -14: 0,
        15: -1, -15: 1, 16: 0, -16: 0,
        # Quarks
        1: -1/3, -1: 1/3, 2: 2/3, -2: -2/3,
        3: -1/3, -3: 1/3, 4: 2/3, -4: -2/3,
        5: -1/3, -5: 1/3, 6: 2/3, -6: -2/3,
        # Light mesons
        111: 0, 211: 1, -211: -1, 113: 0, 213: 1, -213: -1,
        221: 0, 331: 0, 130: 0, 310: 0, 311: 0, -311: 0,
        321: 1, -321: -1,
        # Charmed mesons
        411: 1, -411: -1, 421: 0, -421: 0,
        # Bottom mesons
        511: 0, -511: 0, 521: 1, -521: -1,
        # Baryons
        2212: 1, -2212: -1, 2112: 0, -2112: 0,
        3122: 0, -3122: 0, 3222: 1, -3222: -1,
        3212: 0, -3212: 0, 3112: -1, -3112: 1,
        3312: -1, -3312: 1, 3322: 0, -3322: 0,
    }

    def __new__(cls) -> 'ParticleChargeRegistry':
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

    def get_charge(self, pdg_id: int) -> Optional[float]:
        if pdg_id in self._CHARGE_MAP:
            return self._CHARGE_MAP[pdg_id]
        if pdg_id < 0 and -pdg_id in self._CHARGE_MAP:
            return -self._CHARGE_MAP[-pdg_id]
        return None

    def get_charge_safe(self, pdg_id: int, default: float = 0.0) -> float:
        charge = self.get_charge(pdg_id)
        return charge if charge is not None else default

print("✓ Particle models defined")

## 3. Domain Models - Peak

In [ ]:
@dataclass(frozen=True)
class Peak:
    """Immutable value object representing a peak in Hough space."""
    x: float
    y: float
    height: float

    def distance_to(self, other: 'Peak') -> float:
        return np.sqrt((self.x - other.x)**2 + (self.y - other.y)**2)

    def is_within_tolerance(self, other: 'Peak', tolerance: float) -> bool:
        return self.distance_to(other) <= tolerance

    def to_tuple(self) -> Tuple[float, float, float]:
        return (self.x, self.y, self.height)


class PeakCollection:
    """Collection of peaks with operations."""

    def __init__(self, peaks: List[Peak] = None):
        self._peaks: List[Peak] = peaks if peaks is not None else []

    def add(self, peak: Peak) -> None:
        self._peaks.append(peak)

    def __len__(self) -> int:
        return len(self._peaks)

    def __iter__(self):
        return iter(self._peaks)

    def __getitem__(self, index: int) -> Peak:
        return self._peaks[index]

    def filter_by_height(self, min_height: float) -> 'PeakCollection':
        filtered = [p for p in self._peaks if p.height >= min_height]
        return PeakCollection(filtered)

    def find_within_tolerance(self, target: Peak, tolerance: float) -> List[Peak]:
        return [p for p in self._peaks if target.is_within_tolerance(p, tolerance)]

    def to_array(self) -> np.ndarray:
        if not self._peaks:
            return np.empty((0, 3))
        return np.array([p.to_tuple() for p in self._peaks])

    @classmethod
    def from_tuples(cls, tuples: List[Tuple[float, float, float]]) -> 'PeakCollection':
        peaks = [Peak(x=t[0], y=t[1], height=t[2]) for t in tuples]
        return cls(peaks)

print("✓ Peak models defined")

## 4. Domain Models - Track

In [ ]:
@dataclass
class Track:
    """Represents a track in the detector."""
    event_id: int
    phi_bin: float
    curv_bin: float
    eta: float
    vz: float
    number_of_hits: int
    pz_over_pt: float
    particle_type: int
    phi: float
    pt: float
    pz: float
    is_reconstructed: bool = False

    @property
    def cotangent(self) -> float:
        return self.pz_over_pt

    def is_in_vz_range(self, vz_min: float, vz_max: float) -> bool:
        return vz_min < self.vz < vz_max

    def is_in_cot_range(self, cot_min: float, cot_max: float) -> bool:
        return cot_min < self.cotangent < cot_max

    def mark_reconstructed(self) -> None:
        self.is_reconstructed = True

    def to_dict(self) -> dict:
        return {
            'event_id': self.event_id,
            'phi_bin': self.phi_bin,
            'curv_bin': self.curv_bin,
            'eta': self.eta,
            'vz': self.vz,
            'number_of_hits': self.number_of_hits,
            'pz_over_pt': self.pz_over_pt,
            'particle_type': self.particle_type,
            'phi': self.phi,
            'pt': self.pt,
            'pz': self.pz,
            'reco': 1 if self.is_reconstructed else 0,
        }


class TrackCollection:
    """Collection of tracks with query and filtering operations."""

    def __init__(self, tracks: List[Track] = None):
        self._tracks: List[Track] = tracks if tracks is not None else []
        self._by_event: dict = {}
        self._rebuild_index()

    def _rebuild_index(self) -> None:
        self._by_event = {}
        for track in self._tracks:
            if track.event_id not in self._by_event:
                self._by_event[track.event_id] = []
            self._by_event[track.event_id].append(track)

    def add(self, track: Track) -> None:
        self._tracks.append(track)
        if track.event_id not in self._by_event:
            self._by_event[track.event_id] = []
        self._by_event[track.event_id].append(track)

    def __len__(self) -> int:
        return len(self._tracks)

    def __iter__(self):
        return iter(self._tracks)

    def filter_by_vz_range(self, vz_min: float, vz_max: float) -> 'TrackCollection':
        filtered = [t for t in self._tracks if t.is_in_vz_range(vz_min, vz_max)]
        return TrackCollection(filtered)

    def filter_by_cot_range(self, cot_min: float, cot_max: float) -> 'TrackCollection':
        filtered = [t for t in self._tracks if t.is_in_cot_range(cot_min, cot_max)]
        return TrackCollection(filtered)

    def get_reconstructed(self) -> 'TrackCollection':
        filtered = [t for t in self._tracks if t.is_reconstructed]
        return TrackCollection(filtered)

    def count_reconstructed(self) -> int:
        return sum(1 for t in self._tracks if t.is_reconstructed)

    def to_dataframe(self) -> pd.DataFrame:
        if not self._tracks:
            return pd.DataFrame()
        return pd.DataFrame([t.to_dict() for t in self._tracks])

print("✓ Track models defined")

## 5. Domain Models - Hough Square

In [ ]:
class SquareClassification(Enum):
    """Classification of Hough squares."""
    TRUE_POSITIVE = "true_positive"
    FALSE_POSITIVE = "false_positive"
    UNCLASSIFIED = "unclassified"


@dataclass(frozen=True)
class HoughSquare:
    """Immutable value object representing a square region in Hough space."""
    data: np.ndarray
    classification: SquareClassification
    center_x: float
    center_y: float

    @property
    def is_true_positive(self) -> bool:
        return self.classification == SquareClassification.TRUE_POSITIVE

    @property
    def is_false_positive(self) -> bool:
        return self.classification == SquareClassification.FALSE_POSITIVE


class HoughSquareCollection:
    """Collection of Hough squares for training data."""

    def __init__(self):
        self._true_positives: List[HoughSquare] = []
        self._false_positives: List[HoughSquare] = []

    def add_square(self, square: HoughSquare) -> None:
        if square.is_true_positive:
            self._true_positives.append(square)
        elif square.is_false_positive:
            self._false_positives.append(square)

    @property
    def true_positive_count(self) -> int:
        return len(self._true_positives)

    @property
    def false_positive_count(self) -> int:
        return len(self._false_positives)

    @property
    def total_count(self) -> int:
        return self.true_positive_count + self.false_positive_count

    def get_training_data(self) -> tuple:
        if self.total_count == 0:
            return np.empty((0, 0)), np.empty(0)

        true_data = np.stack([sq.data for sq in self._true_positives], axis=0)
        true_labels = np.ones(len(self._true_positives))

        false_data = np.stack([sq.data for sq in self._false_positives], axis=0)
        false_labels = np.zeros(len(self._false_positives))

        X = np.vstack([true_data, false_data])
        y = np.hstack([true_labels, false_labels])

        return X, y

    def summary(self) -> dict:
        return {
            'true_positives': self.true_positive_count,
            'false_positives': self.false_positive_count,
            'total': self.total_count,
            'true_positive_ratio': (
                self.true_positive_count / self.total_count
                if self.total_count > 0 else 0.0
            )
        }

print("✓ Hough square models defined")

## 6. Configuration Classes

In [ ]:
@dataclass(frozen=True)
class HoughConfig:
    """Configuration for Hough transform parameters."""
    nbin_phi: int = 7000
    nbin_qpt: int = 216
    square_size: int = 16
    tolerance: float = 6.0


@dataclass(frozen=True)
class PeakDetectionConfig:
    """Configuration for peak detection parameters."""
    threshold_abs: float = 5.0
    threshold_rel: float = 0.0
    min_distance: int = 2
    smooth_sigma: float = 0.0


@dataclass(frozen=True)
class VisualizationConfig:
    """Configuration for visualization parameters."""
    start_phi: int = 1000
    end_phi: int = 2000
    size_true: int = 3
    enabled: bool = True
    max_plots: int = 200


@dataclass(frozen=True)
class ProcessingConfig:
    """Configuration for data processing."""
    slice_list: List[int] = field(default_factory=lambda: [-1])
    num_files: int = 8
    min_hits: int = 4
    vz_range: tuple = (-200, 200)


@dataclass(frozen=True)
class PathConfig:
    """Configuration for file paths."""
    data_path: str = "/eos/user/t/tbold/EFTracking/HoughML/pg_2mu_pu100_insquare"
    output_dir: str = "."

    @property
    def data_path_obj(self) -> Path:
        return Path(self.data_path)

    @property
    def output_dir_obj(self) -> Path:
        return Path(self.output_dir)


@dataclass(frozen=True)
class AnalysisConfig:
    """Complete analysis configuration."""
    hough: HoughConfig
    peak_detection: PeakDetectionConfig
    visualization: VisualizationConfig
    processing: ProcessingConfig
    paths: PathConfig
    easing_type: str = "InSquare"


class AnalysisConfigBuilder:
    """Builder for creating AnalysisConfig instances."""

    def __init__(self):
        self._hough: Optional[HoughConfig] = None
        self._peak_detection: Optional[PeakDetectionConfig] = None
        self._visualization: Optional[VisualizationConfig] = None
        self._processing: Optional[ProcessingConfig] = None
        self._paths: Optional[PathConfig] = None
        self._easing_type: str = "InSquare"

    def with_paths(self, data_path: str, output_dir: str = ".") -> 'AnalysisConfigBuilder':
        self._paths = PathConfig(data_path=data_path, output_dir=output_dir)
        return self

    def with_processing_config(self, slice_list: List[int] = None, num_files: int = 8) -> 'AnalysisConfigBuilder':
        if slice_list is None:
            slice_list = [-1]
        self._processing = ProcessingConfig(slice_list=slice_list, num_files=num_files)
        return self

    def build(self) -> AnalysisConfig:
        hough = self._hough or HoughConfig()
        peak_detection = self._peak_detection or PeakDetectionConfig()
        visualization = self._visualization or VisualizationConfig()
        processing = self._processing or ProcessingConfig()
        paths = self._paths or PathConfig()

        return AnalysisConfig(
            hough=hough,
            peak_detection=peak_detection,
            visualization=visualization,
            processing=processing,
            paths=paths,
            easing_type=self._easing_type
        )

    @classmethod
    def from_legacy_config(cls) -> AnalysisConfig:
        return cls().build()

print("✓ Configuration classes defined")

## 7. Strategy Pattern - Easing Functions

In [ ]:
class EasingStrategy(ABC):
    """Abstract base class for easing strategies."""

    @abstractmethod
    def ease(self, x: float) -> float:
        pass

    @abstractmethod
    def name(self) -> str:
        pass


class LinearEasing(EasingStrategy):
    def ease(self, x: float) -> float:
        return x
    def name(self) -> str:
        return "Linear"


class InSineEasing(EasingStrategy):
    def ease(self, x: float) -> float:
        sign = 1 if x > 0 else (-1 if x < 0 else 0)
        return sign * 32 * (1 - math.cos((x * math.pi) / 64))
    def name(self) -> str:
        return "InSine"


class InSquareEasing(EasingStrategy):
    def ease(self, x: float) -> float:
        sign = 1 if x > 0 else (-1 if x < 0 else 0)
        return sign * 32 * (x * x / 1024)
    def name(self) -> str:
        return "InSquare"


class InCubicEasing(EasingStrategy):
    def ease(self, x: float) -> float:
        return 32 * (x * x * x / 32768)
    def name(self) -> str:
        return "InCubic"


class InCircEasing(EasingStrategy):
    def ease(self, x: float) -> float:
        sign = 1 if x > 0 else (-1 if x < 0 else 0)
        return sign * (32 - math.sqrt(1024 - x * x))
    def name(self) -> str:
        return "InCirc"


class EasingStrategyFactory:
    """Factory for creating easing strategy instances."""
    
    _strategies: Dict[str, Type[EasingStrategy]] = {
        "Linear": LinearEasing,
        "InSine": InSineEasing,
        "InSquare": InSquareEasing,
        "InCubic": InCubicEasing,
        "InCirc": InCircEasing,
    }

    @classmethod
    def create(cls, strategy_name: str) -> EasingStrategy:
        strategy_class = cls._strategies.get(strategy_name)
        if strategy_class is None:
            raise ValueError(f"Unknown easing strategy: {strategy_name}")
        return strategy_class()

print("✓ Strategy pattern defined")

## 8. Repository - Particle Data

In [ ]:
class ParticleRepository:
    """Repository for accessing particle simulation data from ROOT files."""

    def __init__(self, data_path: Path):
        self._data_path = data_path
        self._charge_registry = ParticleChargeRegistry()
        self._cached_df: pd.DataFrame = None

    def load_particles(self, pattern: str = "particles*.root") -> pd.DataFrame:
        root_files = sorted(self._data_path.glob(pattern))
        if not root_files:
            raise FileNotFoundError(f"No files matching '{pattern}' found in {self._data_path}")

        print(f"Found {len(root_files)} ROOT files in {self._data_path}\n")

        for file in root_files:
            print(f"📁 {file.name}")
            tree = uproot.open(file)["particles"]
            arrays = tree.arrays(library="np")

        df = pd.DataFrame(arrays)
        self._cached_df = df
        print("\nColumn names:")
        print(list(df.columns))
        return df

    def get_tracks_for_event(self, event_id: int, df: pd.DataFrame = None,
                             nbin_phi: int = 7000, nbin_qpt: int = 216,
                             min_hits: int = 4) -> TrackCollection:
        if df is None:
            df = self._cached_df
        if df is None:
            raise ValueError("No data loaded. Call load_particles() first.")

        tracks = []
        for row in df.itertuples(index=False):
            if row.event_id != event_id:
                continue

            charges = self._get_charges(row.particle_type)
            mask = (np.array(charges) != 0) & (np.array(row.number_of_hits) > min_hits)

            if not np.any(mask):
                continue

            phi = row.phi
            phi_bin = (phi + np.pi) * nbin_phi / (2.0 * np.pi)
            curv = np.array(charges) / row.pt
            curv_bin = int(nbin_qpt / 2.0) + curv * int(nbin_qpt / 2.0)

            for i in np.where(mask)[0]:
                track = Track(
                    event_id=event_id,
                    phi_bin=float(phi_bin[i]),
                    curv_bin=float(curv_bin[i]),
                    eta=float(row.eta[i]),
                    vz=float(row.vz[i]),
                    number_of_hits=int(row.number_of_hits[i]),
                    pz_over_pt=float(row.pz[i] / row.pt[i]),
                    particle_type=int(row.particle_type[i]),
                    phi=float(row.phi[i]),
                    pt=float(row.pt[i]),
                    pz=float(row.pz[i]),
                    is_reconstructed=False
                )
                tracks.append(track)

        return TrackCollection(tracks)

    def get_all_events(self, df: pd.DataFrame = None) -> List[int]:
        if df is None:
            df = self._cached_df
        if df is None:
            raise ValueError("No data loaded. Call load_particles() first.")
        return df["event_id"].unique().tolist()

    def create_tracks_dict(self, nbin_phi: int = 7000, nbin_qpt: int = 216,
                          min_hits: int = 4) -> dict:
        if self._cached_df is None:
            raise ValueError("No data loaded. Call load_particles() first.")

        tracks_dict = {}
        event_ids = self.get_all_events()

        for event_id in event_ids:
            tracks_dict[event_id] = self.get_tracks_for_event(
                event_id, self._cached_df, nbin_phi, nbin_qpt, min_hits
            )

        return tracks_dict

    def _get_charges(self, particle_types) -> List[float]:
        charges = []
        for pid in particle_types:
            charge = self._charge_registry.get_charge_safe(pid)
            charges.append(charge)
        return charges

    def save_tracks_to_root(self, tracks_dict: dict, event_list: List[int],
                           filename: str, treename: str = "ntuple") -> pd.DataFrame:
        combined_dfs = []
        for event_id in event_list:
            if event_id in tracks_dict:
                collection = tracks_dict[event_id]
                df = collection.to_dataframe()
                if not df.empty:
                    combined_dfs.append(df)

        if not combined_dfs:
            raise ValueError("No tracks to save")

        combined_df = pd.concat(combined_dfs, ignore_index=True)
        print(f"Combined DataFrame shape: {combined_df.shape}")

        with uproot.recreate(filename) as file:
            file[treename] = combined_df.to_dict(orient='list')

        print(f"Successfully saved to {filename}")
        return combined_df

print("✓ Particle repository defined")

## 9. Repository - Hough Data

In [ ]:
class HoughHistogram:
    """Value object representing a Hough histogram."""

    def __init__(self, name: str, histogram, event_id: int, slice_num: int):
        self.name = name
        self.histogram = histogram
        self.event_id = event_id
        self.slice_num = slice_num


class HoughRepository:
    """Repository for accessing Hough accumulator data from ROOT files."""

    def __init__(self, data_path: Path):
        self._data_path = data_path

    def find_files(self, pattern: str = "out*.root") -> List[Path]:
        root_files = sorted(self._data_path.glob(pattern))
        print(f"Found {len(root_files)} ROOT files in {self._data_path}\n")
        return root_files

    def iter_histograms(self, file_path: Path) -> Iterator[HoughHistogram]:
        with uproot.open(file_path) as f:
            for key in f.keys():
                obj = f[key]
                if not obj.classname.startswith("TH2"):
                    continue

                event_id, slice_num = self._parse_histogram_name(key)
                yield HoughHistogram(
                    name=key,
                    histogram=obj,
                    event_id=event_id,
                    slice_num=slice_num
                )

    def _parse_histogram_name(self, name: str) -> Tuple[int, int]:
        parts = name.replace('_', ';').split(';')
        return int(parts[1]), int(parts[2])

print("✓ Hough repository defined")

## 10. Repository - Training Data

In [ ]:
class TrainingDataRepository:
    """Repository for saving and loading training data."""

    def __init__(self, output_dir: Path):
        self._output_dir = output_dir
        self._output_dir.mkdir(parents=True, exist_ok=True)

    def save_squares(self, collection: HoughSquareCollection,
                     filename: str = "images.npz") -> None:
        X, y = collection.get_training_data()
        file_path = self._output_dir / filename
        np.savez(file_path, X=X, y=y)

        summary = collection.summary()
        print(f"\nTraining data saved to {file_path}")
        print(f"True positives: {summary['true_positives']}")
        print(f"False positives: {summary['false_positives']}")
        print(f"Total samples: {summary['total']}")
        print(f"Array shapes: X={X.shape}, y={y.shape}")

print("✓ Training data repository defined")

## 11. Service - Peak Detector

In [ ]:
class PeakDetectorService:
    """Service for detecting peaks in 2D Hough accumulator."""

    def __init__(self, config: PeakDetectionConfig):
        self._config = config

    def find_peaks(self, histogram) -> PeakCollection:
        values, xedges, yedges = histogram.to_numpy()
        values = values.T
        values = np.nan_to_num(values, nan=0.0, posinf=0.0, neginf=0.0)

        if self._config.smooth_sigma > 0:
            values = gaussian_filter(values, sigma=self._config.smooth_sigma)

        coords = self._vectorized_sliding_peaks(
            values,
            min_height=self._config.threshold_abs,
            window_size=2 * self._config.min_distance
        )

        xcenters = 0.5 * (xedges[1:] + xedges[:-1])
        ycenters = 0.5 * (yedges[1:] + yedges[:-1])

        peaks = []
        for (y_idx, x_idx) in coords:
            peak = Peak(
                x=float(xcenters[x_idx]),
                y=float(ycenters[y_idx]),
                height=float(values[y_idx, x_idx])
            )
            peaks.append(peak)

        del xedges, yedges, xcenters, ycenters, coords
        gc.collect()

        return PeakCollection(peaks)

    def _vectorized_sliding_peaks(self, matrix: np.ndarray,
                                   min_height: float = 6,
                                   window_size: int = 5) -> np.ndarray:
        if window_size % 2 == 0:
            window_size += 1

        half_window = window_size // 2
        windows_2d = sliding_window_view(matrix, (window_size, window_size))

        center_r, center_c = half_window, half_window
        center_values = windows_2d[:, :, center_r, center_c]

        is_peak = np.ones_like(center_values, dtype=bool)
        window_maxima = np.max(windows_2d, axis=(2, 3))
        is_peak &= (center_values == window_maxima)
        is_peak &= (center_values >= min_height)

        peak_rows, peak_cols = np.where(is_peak)
        adjusted_rows = peak_rows + half_window
        adjusted_cols = peak_cols + half_window

        if len(adjusted_rows) > 0:
            peak_coords = np.stack((adjusted_rows, adjusted_cols), axis=1)
            peak_values = matrix[adjusted_rows, adjusted_cols]

            distances = cdist(peak_coords, peak_coords)
            close_pairs = np.where(
                (distances > 0) & (distances <= self._config.min_distance)
            )

            peaks_to_keep = set(range(len(peak_coords)))

            for i, j in zip(close_pairs[0], close_pairs[1]):
                if i < j and i in peaks_to_keep and j in peaks_to_keep:
                    if peak_values[i] >= peak_values[j]:
                        peaks_to_keep.remove(j)
                    else:
                        peaks_to_keep.remove(i)

            peaks_to_keep = sorted(peaks_to_keep)
            merged_rows = adjusted_rows[peaks_to_keep]
            merged_cols = adjusted_cols[peaks_to_keep]

            return np.stack((merged_rows, merged_cols), axis=1)
        else:
            return np.empty((0, 2), dtype=int)

print("✓ Peak detector service defined")

## 12. Service - Track Slicer

In [ ]:
class TrackSlicerService:
    """Service for slicing tracks based on cotangent ranges."""

    def __init__(self, easing_strategy: EasingStrategy,
                 hough_config: HoughConfig,
                 processing_config: ProcessingConfig):
        self._easing = easing_strategy
        self._hough_config = hough_config
        self._processing_config = processing_config

    def get_slice_bounds(self, slice_num: int) -> Tuple[Optional[float], Optional[float]]:
        if slice_num == -1:
            return None, None
        else:
            lo_cot = self._easing.ease(-32.0 + 2.0 * max(slice_num, 0))
            hi_cot = self._easing.ease(-32.0 + 2.0 * min(slice_num + 1, 32))
            return lo_cot, hi_cot

    def filter_tracks_for_slice(self, tracks: TrackCollection,
                                 slice_num: int) -> TrackCollection:
        vz_min, vz_max = self._processing_config.vz_range

        if slice_num > -1:
            lo_cot, _ = self.get_slice_bounds(0)
            _, hi_cot = self.get_slice_bounds(32)

            filtered = tracks.filter_by_vz_range(vz_min, vz_max)
            filtered = filtered.filter_by_cot_range(lo_cot, hi_cot)
        else:
            lo_cot, _ = self.get_slice_bounds(10)
            _, hi_cot = self.get_slice_bounds(22)

            filtered = tracks.filter_by_vz_range(vz_min, vz_max)
            filtered = filtered.filter_by_cot_range(lo_cot, hi_cot)

            size = self._hough_config.square_size
            nbin_qpt = self._hough_config.nbin_qpt

            final_tracks = []
            for track in filtered:
                if (track.curv_bin > 0 + size and
                    track.curv_bin < nbin_qpt - size):
                    final_tracks.append(track)

            filtered = TrackCollection(final_tracks)

        return filtered

    @classmethod
    def create(cls, easing_type: str, hough_config: HoughConfig,
              processing_config: ProcessingConfig) -> 'TrackSlicerService':
        strategy = EasingStrategyFactory.create(easing_type)
        return cls(strategy, hough_config, processing_config)

print("✓ Track slicer service defined")

## 13. Service - Hough Matcher

In [ ]:
class HoughMatcherService:
    """Service for matching peaks with true tracks and extracting squares."""

    def __init__(self, config: HoughConfig):
        self._config = config

    def match_and_extract_squares(
        self,
        accumulator_values: np.ndarray,
        reconstructed_peaks: PeakCollection,
        true_tracks: TrackCollection
    ) -> Tuple[HoughSquareCollection, np.ndarray]:
        square_collection = HoughSquareCollection()
        assignment_mask = np.zeros(len(true_tracks), dtype=int)
        true_tracks_list = list(true_tracks)

        for peak in reconstructed_peaks:
            square, has_match, matched_idx = self._extract_and_classify_square(
                accumulator_values, peak, true_tracks_list, assignment_mask
            )

            if square is not None:
                square_collection.add_square(square)
                if has_match and matched_idx is not None:
                    assignment_mask[matched_idx] = 1

        return square_collection, assignment_mask

    def _extract_and_classify_square(
        self,
        values: np.ndarray,
        peak: Peak,
        true_tracks: list,
        assignment_mask: np.ndarray
    ) -> Tuple[HoughSquare, bool, int]:
        size = self._config.square_size
        tolerance = self._config.tolerance

        square_x = peak.x - size
        square_y = peak.y - size
        center_x = square_x + size
        center_y = square_y + size

        x_start = max(0, int(square_y))
        x_end = min(values.shape[0], int(square_y + 2*size))
        y_start = max(0, int(square_x))
        y_end = min(values.shape[1], int(square_x + 2*size))

        if x_end <= x_start or y_end <= y_start:
            return None, False, None

        square_region = np.copy(values[x_start:x_end, y_start:y_end])

        if square_region.shape != (2*size, 2*size):
            return None, False, None

        has_match = False
        matched_idx = None

        for idx, track in enumerate(true_tracks):
            if assignment_mask[idx] > 0:
                continue

            track_peak = Peak(
                x=track.curv_bin,
                y=track.phi_bin,
                height=0
            )

            if peak.is_within_tolerance(track_peak, tolerance):
                has_match = True
                matched_idx = idx
                break

        classification = (
            SquareClassification.TRUE_POSITIVE if has_match
            else SquareClassification.FALSE_POSITIVE
        )

        square = HoughSquare(
            data=square_region.astype(int),
            classification=classification,
            center_x=center_x,
            center_y=center_y
        )

        return square, has_match, matched_idx

    def update_track_reconstruction_status(
        self,
        tracks: TrackCollection,
        assignment_mask: np.ndarray
    ) -> int:
        newly_reconstructed = 0
        for idx, track in enumerate(tracks):
            if assignment_mask[idx] > 0 and not track.is_reconstructed:
                track.mark_reconstructed()
                newly_reconstructed += 1
        return newly_reconstructed

print("✓ Hough matcher service defined")

## 14. Service - Visualizer

In [ ]:
class VisualizerService:
    """Service for visualizing Hough accumulator with peaks."""

    def __init__(self, hough_config: HoughConfig, vis_config: VisualizationConfig):
        self._hough_config = hough_config
        self._vis_config = vis_config
        self._plot_count = 0

    def draw_hough_accumulator(
        self,
        values: np.ndarray,
        reconstructed_peaks: PeakCollection,
        true_tracks: TrackCollection,
        slice_num: int
    ) -> None:
        if not self._vis_config.enabled:
            return

        if self._plot_count >= self._vis_config.max_plots:
            return

        start_phi = self._vis_config.start_phi
        end_phi = self._vis_config.end_phi
        size_reco = self._hough_config.square_size
        size_true = self._vis_config.size_true

        fig, ax = plt.subplots(figsize=(16, 24))
        im = ax.imshow(
            values[start_phi:end_phi, :],
            cmap='viridis',
            interpolation='nearest'
        )

        ax.set_aspect('auto')
        plt.colorbar(im, label="Counts")
        ax.set_xlabel("q/pT")
        ax.set_ylabel("phi")
        ax.set_title(f"Hough Accumulator - Slice {slice_num}")

        for peak in reconstructed_peaks:
            square_x = peak.x - size_reco
            square_y = peak.y - start_phi - size_reco

            rect = patches.Rectangle(
                (square_x, square_y),
                2*size_reco, 2*size_reco,
                linewidth=3,
                edgecolor='red',
                facecolor='none'
            )
            ax.add_patch(rect)

        xmin, xmax = ax.get_xlim()
        ymax, ymin = ax.get_ylim()

        for track in true_tracks:
            square_x = int(track.curv_bin) - size_true
            square_y = int(track.phi_bin) - start_phi - size_true

            rect = patches.Rectangle(
                (square_x, square_y),
                2*size_true, 2*size_true,
                linewidth=3,
                edgecolor="yellow",
                facecolor='none'
            )
            ax.add_patch(rect)

            if xmin < square_x < xmax and ymin < square_y < ymax:
                ax.text(
                    square_x, square_y,
                    f"{track.number_of_hits} {round(track.pz_over_pt, 1)}",
                    fontsize=10,
                    color='white',
                    fontweight='bold'
                )

        plt.show()
        self._plot_count += 1

        del values
        gc.collect()

print("✓ Visualizer service defined")

## 15. Utils - Progress Observers

In [ ]:
@dataclass
class AnalysisStatistics:
    """Value object for analysis statistics."""
    total_files: int = 0
    processed_files: int = 0
    total_events: int = 0
    processed_histograms: int = 0
    true_tracks_total: int = 0
    true_squares: int = 0
    false_squares: int = 0
    start_time: datetime = None
    event_ids: List[int] = field(default_factory=list)

    def add_event(self, event_id: int) -> None:
        if event_id not in self.event_ids:
            self.event_ids.append(event_id)
            self.total_events += 1

    def add_true_tracks(self, count: int) -> None:
        self.true_tracks_total += count

    def add_squares(self, true_count: int, false_count: int) -> None:
        self.true_squares += true_count
        self.false_squares += false_count

    def increment_files(self) -> None:
        self.processed_files += 1

    def increment_histograms(self) -> None:
        self.processed_histograms += 1

    @property
    def total_squares(self) -> int:
        return self.true_squares + self.false_squares

    @property
    def reconstruction_efficiency(self) -> float:
        if self.true_tracks_total == 0:
            return 0.0
        return self.true_squares / self.true_tracks_total


class ProgressObserver(ABC):
    """Abstract base class for progress observers."""

    @abstractmethod
    def on_file_start(self, filename: str, file_num: int, total_files: int) -> None:
        pass

    @abstractmethod
    def on_file_complete(self, filename: str, stats: AnalysisStatistics) -> None:
        pass

    @abstractmethod
    def on_event_discovered(self, event_id: int) -> None:
        pass

    @abstractmethod
    def on_slice_start(self, event_id: int, slice_num: int) -> None:
        pass

    @abstractmethod
    def on_slice_complete(
        self,
        event_id: int,
        slice_num: int,
        peaks_found: int,
        true_tracks_in_slice: int,
        matched_tracks: int
    ) -> None:
        pass

    @abstractmethod
    def on_analysis_start(self, total_files: int) -> None:
        pass

    @abstractmethod
    def on_analysis_complete(self, stats: AnalysisStatistics) -> None:
        pass


class ConsoleProgressObserver(ProgressObserver):
    """Console-based progress observer."""

    def __init__(self, verbose: bool = True):
        self._verbose = verbose

    def on_file_start(self, filename: str, file_num: int, total_files: int) -> None:
        print(f"\n{'='*80}")
        print(f"File {file_num}/{total_files}: {filename}")
        print(f"{'='*80}")

    def on_file_complete(self, filename: str, stats: AnalysisStatistics) -> None:
        if self._verbose:
            print(f"\n✓ Completed {filename}")

    def on_event_discovered(self, event_id: int) -> None:
        print(f"\nEvent: {event_id}")

    def on_slice_start(self, event_id: int, slice_num: int) -> None:
        if self._verbose:
            print(f"\n####### Slice number: {slice_num}")

    def on_slice_complete(
        self,
        event_id: int,
        slice_num: int,
        peaks_found: int,
        true_tracks_in_slice: int,
        matched_tracks: int
    ) -> None:
        print(f"Peaks found: {peaks_found}")
        print(f"True tracks in slice: {true_tracks_in_slice}")
        print(f"Matched tracks: {matched_tracks}")

    def on_analysis_start(self, total_files: int) -> None:
        print("="*80)
        print("Hough Transform Analysis - Starting Processing")
        print("="*80)
        print(f"\nTotal files to process: {total_files}")

    def on_analysis_complete(self, stats: AnalysisStatistics) -> None:
        print("\n" + "="*80)
        print("Processing Complete!")
        print("="*80)
        print(f"\nSummary:")
        print(f"  - Files processed: {stats.processed_files}/{stats.total_files}")
        print(f"  - Events processed: {stats.total_events}")
        print(f"  - True tracks found: {stats.true_tracks_total}")
        print(f"  - True squares: {stats.true_squares}")
        print(f"  - False squares: {stats.false_squares}")
        print(f"  - Reconstruction efficiency: {stats.reconstruction_efficiency:.2%}")

print("✓ Progress observers defined")

## 16. Facade - Analysis Facade

In [ ]:
class HoughAnalysisFacade:
    """Facade providing simplified API for Hough transform analysis."""

    def __init__(
        self,
        config: AnalysisConfig,
        observer: Optional[ProgressObserver] = None
    ):
        self._config = config
        self._observer = observer or ConsoleProgressObserver(verbose=True)
        self._stats = AnalysisStatistics()

        # Initialize repositories
        self._particle_repo = ParticleRepository(config.paths.data_path_obj)
        self._hough_repo = HoughRepository(config.paths.data_path_obj)
        self._training_repo = TrainingDataRepository(config.paths.output_dir_obj)

        # Initialize services
        self._peak_detector = PeakDetectorService(config.peak_detection)
        self._track_slicer = TrackSlicerService.create(
            config.easing_type,
            config.hough,
            config.processing
        )
        self._hough_matcher = HoughMatcherService(config.hough)
        self._visualizer = VisualizerService(config.hough, config.visualization)

        # Storage for analysis results
        self._tracks_dict = {}
        self._square_collection = HoughSquareCollection()

    def run_complete_analysis(self) -> AnalysisStatistics:
        """Run complete end-to-end analysis."""
        self._stats.start_time = datetime.now()

        # Step 1: Load particle data
        print("\n[1/4] Loading particle simulation data...")
        df = self._particle_repo.load_particles()

        # Step 2: Create true tracks dictionary
        print("\n[2/4] Creating true tracks dictionary...")
        self._tracks_dict = self._particle_repo.create_tracks_dict(
            nbin_phi=self._config.hough.nbin_phi,
            nbin_qpt=self._config.hough.nbin_qpt,
            min_hits=self._config.processing.min_hits
        )
        print(f"Created tracks for {len(self._tracks_dict)} events")

        # Step 3: Process Hough files
        print("\n[3/4] Processing Hough accumulator files...")
        self._process_hough_files()

        # Step 4: Save results
        print("\n[4/4] Saving results...")
        self._save_results()

        # Notify completion
        self._observer.on_analysis_complete(self._stats)

        return self._stats

    def _process_hough_files(self) -> None:
        files = self._hough_repo.find_files()
        self._stats.total_files = min(len(files), self._config.processing.num_files)
        self._observer.on_analysis_start(self._stats.total_files)

        for file_idx, file_path in enumerate(files):
            if file_idx >= self._config.processing.num_files:
                break
            self._process_single_file(file_path, file_idx + 1)
            self._stats.increment_files()

    def _process_single_file(self, file_path, file_num: int) -> None:
        self._observer.on_file_start(
            file_path.name,
            file_num,
            self._stats.total_files
        )

        for hough_hist in self._hough_repo.iter_histograms(file_path):
            self._process_histogram(hough_hist)
            self._stats.increment_histograms()
            gc.collect()

        self._observer.on_file_complete(file_path.name, self._stats)

    def _process_histogram(self, hough_hist) -> None:
        event_id = hough_hist.event_id
        slice_num = hough_hist.slice_num

        if event_id not in self._stats.event_ids:
            self._stats.add_event(event_id)
            self._observer.on_event_discovered(event_id)

        if slice_num not in self._config.processing.slice_list:
            return

        self._observer.on_slice_start(event_id, slice_num)

        if event_id not in self._tracks_dict:
            return

        true_tracks = self._tracks_dict[event_id]
        filtered_tracks = self._track_slicer.filter_tracks_for_slice(
            true_tracks,
            slice_num
        )

        self._stats.add_true_tracks(len(filtered_tracks))

        peaks = self._peak_detector.find_peaks(hough_hist.histogram)
        values, _, _ = hough_hist.histogram.to_numpy()
        values = values.T

        squares, assignment_mask = self._hough_matcher.match_and_extract_squares(
            values,
            peaks,
            filtered_tracks
        )

        matched_count = self._hough_matcher.update_track_reconstruction_status(
            filtered_tracks,
            assignment_mask
        )

        for square in [s for s in [squares._true_positives, squares._false_positives] for s in s]:
            self._square_collection.add_square(square)

        self._stats.add_squares(
            squares.true_positive_count,
            squares.false_positive_count
        )

        self._visualizer.draw_hough_accumulator(
            values,
            peaks,
            filtered_tracks,
            slice_num
        )

        self._observer.on_slice_complete(
            event_id,
            slice_num,
            len(peaks),
            len(filtered_tracks),
            matched_count
        )

        del values
        gc.collect()

    def _save_results(self) -> None:
        print("\nSaving training data...")
        self._training_repo.save_squares(
            self._square_collection,
            filename="images.npz"
        )

        print("\nSaving true tracks to ROOT file...")
        self._particle_repo.save_tracks_to_root(
            self._tracks_dict,
            self._stats.event_ids,
            filename=str(self._config.paths.output_dir_obj / "out_true_tracks.root"),
            treename="ntuple"
        )

    @classmethod
    def create_with_default_config(cls) -> 'HoughAnalysisFacade':
        config = AnalysisConfigBuilder.from_legacy_config()
        return cls(config)

print("✓ Analysis facade defined")

In [ ]:
# Check current working directory and set output path
import os

# Get current working directory
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")
print(f"Notebook location: {os.path.dirname(os.path.abspath('analyze_ttbar_OOP.ipynb')) if os.path.exists('analyze_ttbar_OOP.ipynb') else 'unknown'}")

# Set your desired output directory here
# Option 1: Use current directory
OUTPUT_DIR = current_dir

# Option 2: Use specific directory (uncomment and modify as needed)
# OUTPUT_DIR = "/path/to/your/output/directory"

# Option 3: Use same directory as notebook (if running from notebook dir)
# OUTPUT_DIR = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else current_dir

print(f"\nOutput files will be saved to: {OUTPUT_DIR}")
print(f"  - images.npz")
print(f"  - out_true_tracks.root")

In [ ]:
# Configure the analysis
config = (AnalysisConfigBuilder()
          .with_paths(
              data_path="/eos/user/t/tbold/EFTracking/HoughML/pg_2mu_pu100_insquare",
              output_dir=OUTPUT_DIR  # Use the OUTPUT_DIR from previous cell
          )
          .with_processing_config(
              slice_list=[-1],
              num_files=8
          )
          .build())

# Print configuration
print("Configuration:")
print(f"  Data path: {config.paths.data_path}")
print(f"  Output dir: {config.paths.output_dir}")
print(f"  Absolute output path: {config.paths.output_dir_obj.absolute()}")
print(f"  Number of files: {config.processing.num_files}")
print(f"  Slice list: {config.processing.slice_list}")
print(f"  Easing type: {config.easing_type}")

In [ ]:
# Create facade and run analysis
facade = HoughAnalysisFacade(config)
stats = facade.run_complete_analysis()

print("\n" + "="*80)
print("✓ Analysis completed successfully!")
print("="*80)

## 18. View Results

Examine the analysis results:

In [ ]:
# Load saved training data
import os
npz_file = os.path.join(OUTPUT_DIR, 'images.npz')

print(f"Loading training data from: {npz_file}")
if not os.path.exists(npz_file):
    print(f"ERROR: File not found! Looking in: {npz_file}")
    print(f"Files in output directory:")
    for f in os.listdir(OUTPUT_DIR):
        print(f"  - {f}")
else:
    data = np.load(npz_file)
    X = data['X']
    y = data['y']

    print(f"\nTraining data loaded successfully!")
    print(f"  Features (X): {X.shape}")
    print(f"  Labels (y): {y.shape}")
    print(f"  True positives: {np.sum(y == 1)}")
    print(f"  False positives: {np.sum(y == 0)}")

    # Show example squares
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    fig.suptitle('Sample Training Data (Top: True Positives, Bottom: False Positives)', fontsize=14)

    # Show true positives
    true_indices = np.where(y == 1)[0][:5]
    for i, idx in enumerate(true_indices):
        axes[0, i].imshow(X[idx], cmap='viridis')
        axes[0, i].set_title(f'True #{idx}')
        axes[0, i].axis('off')

    # Show false positives
    false_indices = np.where(y == 0)[0][:5]
    for i, idx in enumerate(false_indices):
        axes[1, i].imshow(X[idx], cmap='viridis')
        axes[1, i].set_title(f'False #{idx}')
        axes[1, i].axis('off')

    plt.tight_layout()
    plt.show()

## Summary

This notebook contains the complete OOP refactored Hough Transform Analysis system.

**Key Features:**
- ✅ Domain-Driven Design with clear business entities
- ✅ SOLID principles throughout
- ✅ 8 design patterns (Facade, Repository, Strategy, Builder, Observer, Singleton, Factory, Value Object)
- ✅ Fully testable and maintainable
- ✅ Production-ready code quality

**To use:** Simply run all cells, or modify the configuration in cell 17 before running the analysis.

**Output files:**
- `images.npz` - Training data (X and y arrays)
- `out_true_tracks.root` - ROOT file with true track information